In [70]:
#imports
import pandas as pd
import numpy as np
import seaborn as sns

#### Grouping based on the food pyramid in the mid-term delieverable

In [71]:
# Load NEVO data
data = pd.read_csv("NEVO2023_8.0.csv", delimiter="|")

In [72]:
# Reduce to relevant variables
variables = [
    'NEVO-code', 'Food group', 'Engelse naam/Food name', 'Hoeveelheid/Quantity',
    'FAT (g)', 'PROT (g)', 'ENERCC (kcal)', 'THIA (mg)', 'RIBF (mg)', 'VITB6 (mg)', 
    'VITB12 (µg)', 'VITD (µg)', 'VITE (mg)', 'VITC (mg)', 'VITA_RE (µg)', 
    'CA (mg)', 'FE (mg)', 'MG (mg)', 'P (mg)', 'K (mg)', 'NA (mg)', 'ZN (mg)'
]
clean_data = data[variables].copy()

# === Main Wheel of Five Group Mapping ===
wheel_mapping = {
    # Drinks
    'Non-alcoholic beverages': 'Drinks',
    'Alcoholic beverages': 'Other',

    # Vegetables and fruit
    'Vegetables': 'Vegetables and fruit',
    'Fruits': 'Vegetables and fruit',

    # Bread, Grain/Cereal Products and Potatoes
    'Cereals and cereal products': 'Bread, grain/cereal products and potatoes',
    'Bread': 'Bread, grain/cereal products and potatoes',
    'Potatoes and tubers': 'Bread, grain/cereal products and potatoes',

    # Dairy, Nuts, Fish, Legumes, Meat and Eggs
    'Milk and milk products': 'Dairy, nuts, fish, legumes, meat and eggs',
    'Cheese': 'Dairy, nuts, fish, legumes, meat and eggs',
    'Nuts and seeds': 'Dairy, nuts, fish, legumes, meat and eggs',
    'Legumes': 'Dairy, nuts, fish, legumes, meat and eggs',
    'Fish, crustacean and shellfish': 'Dairy, nuts, fish, legumes, meat and eggs',
    'Meat and poultry': 'Dairy, nuts, fish, legumes, meat and eggs',
    'Eggs': 'Dairy, nuts, fish, legumes, meat and eggs',
    'Meat substitutes and dairy substitutes': 'Dairy, nuts, fish, legumes, meat and eggs',

    # Spreading and Cooking Fats
    'Fats and oils': 'Spreading and cooking fats',
    'Savoury bread spreads': 'Spreading and cooking fats',

    # Outside the Wheel
    'Pastry and biscuits': 'Other',
    'Savoury snacks': 'Other',
    'Sugar, sweets and sweet sauces': 'Other',
    'Savoury sauces': 'Other',
    'Cold meat cuts': 'Other',
    'Mixed dishes': 'Other',
    'Soups': 'Other',
    'Miscellaneous foods': 'Other',
    'Foods for special nutritional use': 'Other',
    'Herbs and spices': 'Other'
}

clean_data['food_wheel_group'] = clean_data['Food group'].map(wheel_mapping)

# === Subgroup Mapping (matching Voedingscentrum’s Wheel of Five split) ===
subgroup_mapping = {
    # Vegetables and fruit
    'Vegetables': 'Vegetables',
    'Fruits': 'Fruit',

    # Dairy, nuts, fish, legumes, meat and eggs
    'Milk and milk products': 'Dairy',
    'Cheese': 'Dairy',
    'Nuts and seeds': 'Nuts',
    'Legumes': 'Fish, legumes, meat and eggs',
    'Fish, crustacean and shellfish': 'Fish, legumes, meat and eggs',
    'Meat and poultry': 'Fish, legumes, meat and eggs',
    'Eggs': 'Fish, legumes, meat and eggs',
    'Meat substitutes and dairy substitutes': 'Fish, legumes, meat and eggs'
}

clean_data['food_wheel_subgroup'] = clean_data['Food group'].map(subgroup_mapping)
clean_data['food_wheel_subgroup'] = clean_data['food_wheel_subgroup'].fillna(clean_data['food_wheel_group'])

display(clean_data)

,NEVO-code,Food group,Engelse naam/Food name,Hoeveelheid/Quantity,FAT (g),PROT (g),ENERCC (kcal),THIA (mg),RIBF (mg),VITB6 (mg),...,VITA_RE (µg),CA (mg),FE (mg),MG (mg),P (mg),K (mg),NA (mg),ZN (mg),food_wheel_group,food_wheel_subgroup
0,1,Potatoes and tubers,Potatoes raw,per 100g,0,2,88,"0,12","0,04","0,3",...,1.0,6.0,"0,5",22.0,60.0,450.0,2.0,"0,37","Bread, grain/cereal products and potatoes","Bread, grain/cereal products and potatoes"
1,2,Potatoes and tubers,Potatoes new raw,per 100g,0,2,88,"0,12","0,04","0,3",...,0.0,6.0,"0,5",14.0,60.0,450.0,2.0,"0,2","Bread, grain/cereal products and potatoes","Bread, grain/cereal products and potatoes"
2,3,Potatoes and tubers,Potatoes old raw,per 100g,0,2,88,"0,12","0,04","0,22",...,1.0,6.0,"0,5",17.0,60.0,450.0,2.0,"0,3","Bread, grain/cereal products and potatoes","Bread, grain/cereal products and potatoes"
3,4,Cereals and cereal products,Pasta white raw,per 100g,"1,5","12,3",356,"0,1","0,06","0,08",...,0.0,15.0,2,53.0,125.0,100.0,2.0,"1,5","Bread, grain/cereal products and potatoes","Bread, grain/cereal products and potatoes"
4,5,Cereals and cereal products,Rice white raw,per 100g,1,7,352,"0,04","0,03","0,12",...,0.0,10.0,"0,4",13.0,100.0,100.0,2.0,"1,8","Bread, grain/cereal products and potatoes","Bread, grain/cereal products and potatoes"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2318,5573,Meat substitutes and dairy substitutes,Tempeh unprepared,per 100g,"4,7","17,6",128,"0,03","0,18","0,091",...,0.0,88.0,"1,7",60.0,190.0,310.0,39.0,"1,3","Dairy, nuts, fish, legumes, meat and eggs","Fish, legumes, meat and eggs"
2319,5574,Cereals and cereal products,Bakery mix for doughnuts Dutch style,per 100g,1,11,367,"0,09","0,09","0,134",...,0.0,22.0,1,21.0,110.0,167.0,680.0,"0,59","Bread, grain/cereal products and potatoes","Bread, grain/cereal products and potatoes"
2320,5575,Cheese,Cheese 35+,per 100g,"20,4","27,3",302,"0,03","0,27","0,058",...,200.0,959.0,"0,2",36.0,606.0,72.0,661.0,"3,99","Dairy, nuts, fish, legumes, meat and eggs",Dairy
2321,5576,Cereals and cereal products,Flour spelt,per 100g,"1,7","13,2",348,"0,21","0,03","0,088",...,0.0,20.0,"1,6",46.0,180.0,190.0,0.0,"1,5","Bread, grain/cereal products and potatoes","Bread, grain/cereal products and potatoes"


In [73]:
#rename nutri columns 
clean_data.rename(columns={"Engelse naam/Food name": "food_name"}, inplace=True)
clean_data.rename(columns={"ENERCC (kcal)": "Carbohydrates (kcal)"}, inplace=True)
clean_data.rename(columns={"THIA (mg)": "VITB1 (mg)"}, inplace=True)
clean_data.rename(columns={"VITA_RE (µg)": "VITA (µg)"}, inplace=True)
clean_data.rename(columns={"CA (mg)": "Calcium (mg)"}, inplace=True)
clean_data.rename(columns={"FE (mg)": "Iron (mg)"}, inplace=True)
clean_data.rename(columns={"MG (mg)": "Magnesium (mg)"}, inplace=True)
clean_data.rename(columns={"P (mg)": "Phosphorus (mg)"}, inplace=True)
clean_data.rename(columns={"K (mg)": "Potassium (mg)"}, inplace=True)
clean_data.rename(columns={"NA (mg)": "Sodium (mg)"}, inplace=True)
clean_data.rename(columns={"ZN (mg)": "Zinc (mg)"}, inplace=True)

In [74]:
nutrients = [
    'FAT (g)', 'PROT (g)', 'Carbohydrates (kcal)', 'VITB1 (mg)', 'RIBF (mg)', 
    'VITB6 (mg)', 'VITB12 (µg)', 'VITD (µg)', 'VITE (mg)', 'VITC (mg)', 'VITA (µg)',
    'Calcium (mg)', 'Iron (mg)', 'Magnesium (mg)', 'Phosphorus (mg)',
    'Potassium (mg)', 'Sodium (mg)', 'Zinc (mg)'
]

### Convert comma decimal strings to float for all nutrient columns
for col in nutrients:
    clean_data[col] = (clean_data[col].astype(str).str
                       .replace(',', '.', regex=False)
                       .replace('nan', None).astype(float)
    )

In [75]:
#check for missing values
print(clean_data.isnull().sum())

#remove rows with missing values
clean_data = clean_data.dropna()

NEVO-code                 0
Food group                0
food_name                 0
Hoeveelheid/Quantity      0
FAT (g)                   0
PROT (g)                  0
Carbohydrates (kcal)      0
VITB1 (mg)               54
RIBF (mg)                51
VITB6 (mg)               97
VITB12 (µg)              38
VITD (µg)                52
VITE (mg)               246
VITC (mg)                49
VITA (µg)                43
Calcium (mg)             25
Iron (mg)                 9
Magnesium (mg)           98
Phosphorus (mg)          46
Potassium (mg)           31
Sodium (mg)               2
Zinc (mg)               125
food_wheel_group          0
food_wheel_subgroup       0
dtype: int64


In [76]:
clean_data

,NEVO-code,Food group,food_name,Hoeveelheid/Quantity,FAT (g),PROT (g),Carbohydrates (kcal),VITB1 (mg),RIBF (mg),VITB6 (mg),...,VITA (µg),Calcium (mg),Iron (mg),Magnesium (mg),Phosphorus (mg),Potassium (mg),Sodium (mg),Zinc (mg),food_wheel_group,food_wheel_subgroup
0,1,Potatoes and tubers,Potatoes raw,per 100g,0.0,2.0,88.0,0.12,0.04,0.300,...,1.0,6.0,0.5,22.0,60.0,450.0,2.0,0.37,"Bread, grain/cereal products and potatoes","Bread, grain/cereal products and potatoes"
1,2,Potatoes and tubers,Potatoes new raw,per 100g,0.0,2.0,88.0,0.12,0.04,0.300,...,0.0,6.0,0.5,14.0,60.0,450.0,2.0,0.20,"Bread, grain/cereal products and potatoes","Bread, grain/cereal products and potatoes"
2,3,Potatoes and tubers,Potatoes old raw,per 100g,0.0,2.0,88.0,0.12,0.04,0.220,...,1.0,6.0,0.5,17.0,60.0,450.0,2.0,0.30,"Bread, grain/cereal products and potatoes","Bread, grain/cereal products and potatoes"
3,4,Cereals and cereal products,Pasta white raw,per 100g,1.5,12.3,356.0,0.10,0.06,0.080,...,0.0,15.0,2.0,53.0,125.0,100.0,2.0,1.50,"Bread, grain/cereal products and potatoes","Bread, grain/cereal products and potatoes"
4,5,Cereals and cereal products,Rice white raw,per 100g,1.0,7.0,352.0,0.04,0.03,0.120,...,0.0,10.0,0.4,13.0,100.0,100.0,2.0,1.80,"Bread, grain/cereal products and potatoes","Bread, grain/cereal products and potatoes"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2316,5571,Meat substitutes and dairy substitutes,Drink soya chocolate w sugar fortified w calci...,per 100g,1.7,3.0,60.0,0.02,0.20,0.047,...,0.0,115.0,0.5,22.0,87.0,180.0,56.0,0.32,"Dairy, nuts, fish, legumes, meat and eggs","Fish, legumes, meat and eggs"
2317,5572,Meat substitutes and dairy substitutes,Bacon vegetarian unprepared,per 100g,10.5,15.0,185.0,0.04,0.17,0.041,...,0.0,100.0,3.9,12.0,180.0,420.0,1057.0,0.72,"Dairy, nuts, fish, legumes, meat and eggs","Fish, legumes, meat and eggs"
2318,5573,Meat substitutes and dairy substitutes,Tempeh unprepared,per 100g,4.7,17.6,128.0,0.03,0.18,0.091,...,0.0,88.0,1.7,60.0,190.0,310.0,39.0,1.30,"Dairy, nuts, fish, legumes, meat and eggs","Fish, legumes, meat and eggs"
2319,5574,Cereals and cereal products,Bakery mix for doughnuts Dutch style,per 100g,1.0,11.0,367.0,0.09,0.09,0.134,...,0.0,22.0,1.0,21.0,110.0,167.0,680.0,0.59,"Bread, grain/cereal products and potatoes","Bread, grain/cereal products and potatoes"


In [77]:
clean_data.columns

Index(['NEVO-code', 'Food group', 'food_name', 'Hoeveelheid/Quantity',
       'FAT (g)', 'PROT (g)', 'Carbohydrates (kcal)', 'VITB1 (mg)',
       'RIBF (mg)', 'VITB6 (mg)', 'VITB12 (µg)', 'VITD (µg)', 'VITE (mg)',
       'VITC (mg)', 'VITA (µg)', 'Calcium (mg)', 'Iron (mg)', 'Magnesium (mg)',
       'Phosphorus (mg)', 'Potassium (mg)', 'Sodium (mg)', 'Zinc (mg)',
       'food_wheel_group', 'food_wheel_subgroup'],
      dtype='object')

In [78]:
clean_data = clean_data[['NEVO-code', 'food_name',
                         'FAT (g)', 'PROT (g)', 'Carbohydrates (kcal)',
                         'VITB1 (mg)', 'RIBF (mg)', 'VITB6 (mg)', 'VITB12 (µg)', 'VITD (µg)',
                         'VITE (mg)', 'VITC (mg)', 'VITA (µg)', 'Calcium (mg)', 'Iron (mg)',
                         'Magnesium (mg)', 'Phosphorus (mg)', 'Potassium (mg)', 'Sodium (mg)',
                         'Zinc (mg)', 'food_wheel_group', 'food_wheel_subgroup']]

In [79]:
clean_data.to_csv('clean_data.csv', index=False)